# DataFrames

DataFrame is a distributed collection of data organized into named columns.
It is normally an API built on top of RDDs.
DF is conceptually equivalent to a table in a relational database or a data frame in Python, but with richer optimizations under the hood. 
DataFrames can be constructed from a wide array of sources such as:
    structured data files
    tables in Hive
    external databases
    existing RDDs

Rows and Columns in DataFrames are nothing but Row Objects and Column Objects of Class:
    pyspark.sql.Row
    pyspark.sql.Column

# Creating DataFrames

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("DataFrame_basics").getOrCreate()

2021-09-21 08:06:56,686 WARN util.Utils: Your hostname, tb-LinuxBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2021-09-21 08:06:56,688 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
2021-09-21 08:06:59,038 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
columns = ["language","count"]
data = [("Python", 100000),("Java",50000),("Scala",8000)]

Create DataFrame from RDD

In [4]:
dataRDD = spark.sparkContext.parallelize(data)

In [5]:
rddToDF = dataRDD.toDF()

In [6]:
rddToDF

DataFrame[_1: string, _2: bigint]

In [7]:
type(rddToDF)

pyspark.sql.dataframe.DataFrame

In [8]:
rddToDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)



In [9]:
columns = ["language","count"]
rddToDF = dataRDD.toDF(columns)

In [10]:
rddToDF.show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



In [11]:
rddToDF.printSchema()

root
 |-- language: string (nullable = true)
 |-- count: long (nullable = true)



In [12]:
columns = ["language","count"]
rddtoDF2 = spark.createDataFrame(data,schema=columns)

In [13]:
rddtoDF2.show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



In [14]:
rddtoDF2.printSchema()

root
 |-- language: string (nullable = true)
 |-- count: long (nullable = true)



Create DataFrame from List

In [15]:
columns = ["language","count"]
data = [("Python", 100000),("Java",50000),("Scala",8000)]

In [16]:
listToDF = spark.createDataFrame(data,schema=columns)

In [17]:
listToDF.show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



In [18]:
listToDF.printSchema()

root
 |-- language: string (nullable = true)
 |-- count: long (nullable = true)



In [19]:
from pyspark.sql import Row

In [20]:
row_data6 = map(lambda x: Row(x[0],x[1]),data) #returns a map object

In [21]:
row_data6

In [22]:
spark.createDataFrame(data=row_data6,schema=columns).show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



# Create DataFrames with Schema

from pyspark.sql import StructType, StructField

StructType
class pyspark.sql.types.StructType(fields=None)
Struct type, consisting of a list of StructField.

This is the data type representing a Row.

Iterating a StructType will iterate over its StructFields. A contained StructField can be accessed by its name or position.

StructField

class pyspark.sql.types.StructField(name, dataType, nullable=True, metadata=None)[source]

A field in StructType.

Important data types in pyspark.sql.types

IntegerType() |
FloatType() | 
StringType() | 
BooleanType() | 
TimestampType() | 
DateType()

In [23]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [24]:
schema = StructType([\
                     StructField("first_name",StringType(),True)\
                     ,StructField("middle_name",StringType(),True)\
                     ,StructField("last_name",StringType(),False)\
                     ,StructField("id",IntegerType(),False)\
                     ,StructField("gender",StringType(),True)\
                     ,StructField("salary",FloatType(),True)\
                    ])

In [25]:
data = [\
        ("Jonathan","Don","Snow",100,'M',5000.50),\
        ("Tony","","Stark",200,'M',19000.00),\
        ("Arya","","Stark",0,'F',8000.50),\
        ("Bruce","Ben","Banner",400,'M',9000.80),\
        ("Natasha","Wonder","Romanov",500,'F',7000.50),\
        ]

In [26]:
type(schema)

pyspark.sql.types.StructType

In [27]:
df = spark.createDataFrame(data=data,schema=schema)

In [28]:
df.show()

+----------+-----------+---------+---+------+-------+
|first_name|middle_name|last_name| id|gender| salary|
+----------+-----------+---------+---+------+-------+
|  Jonathan|        Don|     Snow|100|     M| 5000.5|
|      Tony|           |    Stark|200|     M|19000.0|
|      Arya|           |    Stark|  0|     F| 8000.5|
|     Bruce|        Ben|   Banner|400|     M| 9000.8|
|   Natasha|     Wonder|  Romanov|500|     F| 7000.5|
+----------+-----------+---------+---+------+-------+



In [29]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = false)
 |-- id: integer (nullable = false)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)



# Create DataFrame from Data Sources

csv, txt, json, parquet

The APIs are present in pyspark.sql.readwriter inside DataFrameReader class:<br>
Important methods from class DataFrameReader:<br>

1. format(self, source) -> Specifies the input data source format<br>
2. schema(self, schema) -> Specifies the input schema<br>
        Some data sources (e.g. JSON) can infer the input schema automatically from data.
        By specifying the schema here, the underlying data source can skip the schema
        inference step, and thus speed up data loading.<br>
3. option(self, key, value) -> Adds an input option for the underlying data source (doubt)<br>
4. load(self, path=None, format=None, schema=None, **options)<br>
        Parameters
        ----------
        path : str or list, optional
            optional string or a list of string for file-system backed data sources.
        format : str, optional
            optional string for format of the data source. Default to 'parquet'.
        schema : :class:`pyspark.sql.types.StructType` or str, optional
            optional :class:`pyspark.sql.types.StructType` for the input schema
            or a DDL-formatted string (For example ``col0 INT, col1 DOUBLE``).
        **options : dict
            all other string options<br>
<br>
5. json(self, path, mode=None, compression=None, dateFormat=None, timestampFormat=None,<br>
                 lineSep=None, encoding=None, ignoreNullFields=None)<br>
                 -> Loads JSON files and returns the results as a :class:`DataFrame`<br>
<br>
6. table(self, tableName) -> Returns the specified table as a :class:`DataFrame`<br>
<br>
7. parquet(self, *paths, **options) -> Loads Parquet files, returning the result as a :class:`DataFrame`<br>
<br>
    important options: recursiveFileLookup<br>
<br>
    modifiedBefore (batch only) : an optional timestamp to only include files with
        modification times occurring before the specified time. The provided timestamp
        must be in the following format: YYYY-MM-DDTHH:mm:ss (e.g. 2020-06-01T13:00:00)
<br>
    modifiedAfter (batch only) : an optional timestamp to only include files with
        modification times occurring after the specified time. The provided timestamp
        must be in the following format: YYYY-MM-DDTHH:mm:ss (e.g. 2020-06-01T13:00:00)<br>
<br>
8. text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,<br>
             recursiveFileLookup=None, modifiedBefore=None,<br>
             modifiedAfter=None) -> <br>
<br>
            Loads text files and returns a :class:`DataFrame` whose schema starts with a
            string column named "value", and followed by partitioned columns if there
            are any.
            The text files must be encoded as UTF-8.
<br>
            By default, each line in the text file is a new row in the resulting DataFrame.<br>
<br>
9. csv(self, path, schema=None, sep=None, encoding=None, quote=None, escape=None,<br>
            comment=None, header=None, inferSchema=None, ignoreLeadingWhiteSpace=None,<br>
            ignoreTrailingWhiteSpace=None, nullValue=None, nanValue=None, positiveInf=None,<br>
            negativeInf=None, dateFormat=None, timestampFormat=None, maxColumns=None,<br>
            maxCharsPerColumn=None, maxMalformedLogPerPartition=None, mode=None,<br>
            columnNameOfCorruptRecord=None, multiLine=None, charToEscapeQuoteEscaping=None,<br>
            samplingRatio=None, enforceSchema=None, emptyValue=None, locale=None, lineSep=None,<br>
            pathGlobFilter=None, recursiveFileLookup=None, modifiedBefore=None, modifiedAfter=None,<br>
            unescapedQuoteHandling=None) <br>
            -> Loads a CSV file and returns the result as a  :class:`DataFrame`<br>
<br>
10. saveAsTable(self, name, format=None, mode=None, partitionBy=None, **options)<br>
            -> Saves the content of the :class:`DataFrame` as the specified table.<br>
                In the case the table already exists, behavior of this function depends on the
                save mode, specified by the `mode` function (default to throwing an exception).
                When `mode` is `Overwrite`, the schema of the :class:`DataFrame` does not need to be
                the same as that of the existing table.<br>

                * `append`: Append contents of this :class:`DataFrame` to existing data.
                * `overwrite`: Overwrite existing data.
                * `error` or `errorifexists`: Throw an exception if data already exists.
                * `ignore`: Silently ignore this operation if data already exists.<br>
<br>
11. partitionBy(self, *cols) ->
            Partitions the output by the given columns on the file system.<br>
<br>
12. bucketBy(self, numBuckets, col, *cols) ->
            Buckets the output by the given columns.If specified,
            the output is laid out on the file system similar to Hive's bucketing scheme.<br>
<br>
13. sortBy(self, col, *cols) ->
            Sorts the output in each bucket by the given columns on the file system.<br>
<br>
14. jdbc(self, url, table, mode=None, properties=None):
        Saves the content of the :class:`DataFrame` to an external database table via JDBC.
<br>

In [30]:
csvDF = spark.read.format("csv").options(header=True).load("/sparkdata/CarPrice_Assignment.csv")

In [31]:
csvDF.show(2)

2021-09-21 08:07:41,900 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+---------+-------------------+--------+----------+----------+-----------+----------+--------------+---------+---------+--------+---------+----------+----------+--------------+----------+----------+---------+------+----------------+----------+-------+-------+----------+-----+
|car_ID|symboling|            CarName|fueltype|aspiration|doornumber|    carbody|drivewheel|enginelocation|wheelbase|carlength|carwidth|carheight|curbweight|enginetype|cylindernumber|enginesize|fuelsystem|boreratio|stroke|compressionratio|horsepower|peakrpm|citympg|highwaympg|price|
+------+---------+-------------------+--------+----------+----------+-----------+----------+--------------+---------+---------+--------+---------+----------+----------+--------------+----------+----------+---------+------+----------------+----------+-------+-------+----------+-----+
|     1|        3| alfa-romero giulia|     gas|       std|       two|convertible|       rwd|         front|     88.6|    168.8|    64.1|     48.8|  

In [32]:
txtDF = spark.read.format("text").options(header=True).load("/sparkdata/word_count_data2.txt")

In [33]:
txtDF.show(2)

+--------------------+
|               value|
+--------------------+
|Now the way that ...|
|that the robbers ...|
+--------------------+
only showing top 2 rows



In [34]:
txtDF = spark.read.format("json").options(header=True).load("/sparkdata/store_locations.json")

In [35]:
txtDF.show(2)

+--------+-----+---------+
|    city|state| zip_code|
+--------+-----+---------+
| Antioch|   CA|945097911|
|Woodland|   CA|957765409|
+--------+-----+---------+
only showing top 2 rows



In [36]:
txtDF = spark.read.format("parquet").options(header=True).load("/sparkdata/parquet_data")

In [37]:
txtDF.show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

# Convert RDD to DF

In [38]:
columns = ["language","count"]
data = [("Python", 100000),("Java",50000),("Scala",8000)]

In [39]:
sampleRDD = spark.sparkContext.parallelize(data)

In [40]:
sampleRDD.collect()

[('Python', 100000), ('Java', 50000), ('Scala', 8000)]

In [41]:
rddToDF = sampleRDD.toDF(schema=columns)

In [42]:
rddToDF.show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



In [43]:
rddToDF2 = spark.createDataFrame(sampleRDD, schema=columns)

In [44]:
rddToDF2.show()

+--------+------+
|language| count|
+--------+------+
|  Python|100000|
|    Java| 50000|
|   Scala|  8000|
+--------+------+



# Select

In [45]:
data = [\
        ("Jonathon","Don","Snow",100,"M",5000.00)\
        ,("Arya","K","Stark",101,"F",10000.00)\
        ,("Steve","S","Rogers",102,"M",8000.00)\
        ,("Natasha","J","Romanov",103,"F",19000.00)\
        ,("Tyrion","K","Lannister",104,"M",11000.00)\
       ]

In [46]:
columns = ["first_name","middle_name","last_name","id","gender","salary"]

In [47]:
df = spark.createDataFrame(data,columns)

In [48]:
df.show()

+----------+-----------+---------+---+------+-------+
|first_name|middle_name|last_name| id|gender| salary|
+----------+-----------+---------+---+------+-------+
|  Jonathon|        Don|     Snow|100|     M| 5000.0|
|      Arya|          K|    Stark|101|     F|10000.0|
|     Steve|          S|   Rogers|102|     M| 8000.0|
|   Natasha|          J|  Romanov|103|     F|19000.0|
|    Tyrion|          K|Lannister|104|     M|11000.0|
+----------+-----------+---------+---+------+-------+



In [49]:
df.select("first_name","salary").show()

+----------+-------+
|first_name| salary|
+----------+-------+
|  Jonathon| 5000.0|
|      Arya|10000.0|
|     Steve| 8000.0|
|   Natasha|19000.0|
|    Tyrion|11000.0|
+----------+-------+



In [50]:
df.select(df.first_name,df.salary).show()

+----------+-------+
|first_name| salary|
+----------+-------+
|  Jonathon| 5000.0|
|      Arya|10000.0|
|     Steve| 8000.0|
|   Natasha|19000.0|
|    Tyrion|11000.0|
+----------+-------+



In [51]:
df.select(df["first_name"],df["salary"]).show()

+----------+-------+
|first_name| salary|
+----------+-------+
|  Jonathon| 5000.0|
|      Arya|10000.0|
|     Steve| 8000.0|
|   Natasha|19000.0|
|    Tyrion|11000.0|
+----------+-------+



In [52]:
from pyspark.sql.functions import col
df.select(col("first_name"),col("salary")).show()

+----------+-------+
|first_name| salary|
+----------+-------+
|  Jonathon| 5000.0|
|      Arya|10000.0|
|     Steve| 8000.0|
|   Natasha|19000.0|
|    Tyrion|11000.0|
+----------+-------+



In [53]:
df.select("*").show()

+----------+-----------+---------+---+------+-------+
|first_name|middle_name|last_name| id|gender| salary|
+----------+-----------+---------+---+------+-------+
|  Jonathon|        Don|     Snow|100|     M| 5000.0|
|      Arya|          K|    Stark|101|     F|10000.0|
|     Steve|          S|   Rogers|102|     M| 8000.0|
|   Natasha|          J|  Romanov|103|     F|19000.0|
|    Tyrion|          K|Lannister|104|     M|11000.0|
+----------+-----------+---------+---+------+-------+



In [54]:
df.select(columns[1:5]).show()

+-----------+---------+---+------+
|middle_name|last_name| id|gender|
+-----------+---------+---+------+
|        Don|     Snow|100|     M|
|          K|    Stark|101|     F|
|          S|   Rogers|102|     M|
|          J|  Romanov|103|     F|
|          K|Lannister|104|     M|
+-----------+---------+---+------+



In [55]:
df.select(columns[2]).show()

+---------+
|last_name|
+---------+
|     Snow|
|    Stark|
|   Rogers|
|  Romanov|
|Lannister|
+---------+



# Collect

In [56]:
df.collect()

[Row(first_name='Jonathon', middle_name='Don', last_name='Snow', id=100, gender='M', salary=5000.0),
 Row(first_name='Arya', middle_name='K', last_name='Stark', id=101, gender='F', salary=10000.0),
 Row(first_name='Steve', middle_name='S', last_name='Rogers', id=102, gender='M', salary=8000.0),
 Row(first_name='Natasha', middle_name='J', last_name='Romanov', id=103, gender='F', salary=19000.0),
 Row(first_name='Tyrion', middle_name='K', last_name='Lannister', id=104, gender='M', salary=11000.0)]

In [57]:
for i in df.collect(): print(i)

Row(first_name='Jonathon', middle_name='Don', last_name='Snow', id=100, gender='M', salary=5000.0)
Row(first_name='Arya', middle_name='K', last_name='Stark', id=101, gender='F', salary=10000.0)
Row(first_name='Steve', middle_name='S', last_name='Rogers', id=102, gender='M', salary=8000.0)
Row(first_name='Natasha', middle_name='J', last_name='Romanov', id=103, gender='F', salary=19000.0)
Row(first_name='Tyrion', middle_name='K', last_name='Lannister', id=104, gender='M', salary=11000.0)


In [58]:
for i in df.collect(): print(i[:])

('Jonathon', 'Don', 'Snow', 100, 'M', 5000.0)
('Arya', 'K', 'Stark', 101, 'F', 10000.0)
('Steve', 'S', 'Rogers', 102, 'M', 8000.0)
('Natasha', 'J', 'Romanov', 103, 'F', 19000.0)
('Tyrion', 'K', 'Lannister', 104, 'M', 11000.0)


# withColumn

Used to change value, convert data type of existing column, create new column

def withColumn(self, colName, col):
        """
        Returns a new :class:`DataFrame` by adding a column or replacing the
        existing column that has the same name.

        The column expression must be an expression over this :class:`DataFrame`; attempting to add
        a column from some other :class:`DataFrame` will raise an error.

        .. versionadded:: 1.3.0

        Parameters
        ----------
        colName : str
            string, name of the new column.
        col : :class:`Column`
            a :class:`Column` expression for the new column.

In [59]:
data = [\
        ("Jonathon","Don","Snow","100","M","5000.00")\
        ,("Arya","K","Stark","101","F","10000.00")\
        ,("Steve","S","Rogers","102","M","8000.00")\
        ,("Natasha","J","Romanov","103","F","19000.00")\
        ,("Tyrion","K","Lannister","104","M","11000.00")\
       ]

In [60]:
columns = ["first_name","middle_name","last_name","id","gender","salary"]

In [61]:
df = spark.createDataFrame(data=data,schema=columns)

In [62]:
df.show()

+----------+-----------+---------+---+------+--------+
|first_name|middle_name|last_name| id|gender|  salary|
+----------+-----------+---------+---+------+--------+
|  Jonathon|        Don|     Snow|100|     M| 5000.00|
|      Arya|          K|    Stark|101|     F|10000.00|
|     Steve|          S|   Rogers|102|     M| 8000.00|
|   Natasha|          J|  Romanov|103|     F|19000.00|
|    Tyrion|          K|Lannister|104|     M|11000.00|
+----------+-----------+---------+---+------+--------+



In [63]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [64]:
from pyspark.sql.types import IntegerType, FloatType

In [65]:
df = df.withColumn("id",df.id.cast(IntegerType()))

In [66]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [67]:
df = df.withColumn("salary",df.salary.cast(FloatType()))

In [68]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: float (nullable = true)



In [69]:
#Increase salary by 20%

In [70]:
df = df.withColumn("salary",df.salary*1.2)

In [71]:
df.show()

+----------+-----------+---------+---+------+-------+
|first_name|middle_name|last_name| id|gender| salary|
+----------+-----------+---------+---+------+-------+
|  Jonathon|        Don|     Snow|100|     M| 6000.0|
|      Arya|          K|    Stark|101|     F|12000.0|
|     Steve|          S|   Rogers|102|     M| 9600.0|
|   Natasha|          J|  Romanov|103|     F|22800.0|
|    Tyrion|          K|Lannister|104|     M|13200.0|
+----------+-----------+---------+---+------+-------+



In [72]:
#Add a column

In [73]:
from pyspark.sql.functions import lit,when

In [74]:
df = df.withColumn(  \
                    "full_gender",\
                    when(df.gender == 'F',lit("Female"))\
                   .when(df.gender == 'M',"Male").otherwise("NA")\
                  )

In [75]:
df.show()

+----------+-----------+---------+---+------+-------+-----------+
|first_name|middle_name|last_name| id|gender| salary|full_gender|
+----------+-----------+---------+---+------+-------+-----------+
|  Jonathon|        Don|     Snow|100|     M| 6000.0|       Male|
|      Arya|          K|    Stark|101|     F|12000.0|     Female|
|     Steve|          S|   Rogers|102|     M| 9600.0|       Male|
|   Natasha|          J|  Romanov|103|     F|22800.0|     Female|
|    Tyrion|          K|Lannister|104|     M|13200.0|       Male|
+----------+-----------+---------+---+------+-------+-----------+



In [76]:
df.drop(df.middle_name).show()

+----------+---------+---+------+-------+-----------+
|first_name|last_name| id|gender| salary|full_gender|
+----------+---------+---+------+-------+-----------+
|  Jonathon|     Snow|100|     M| 6000.0|       Male|
|      Arya|    Stark|101|     F|12000.0|     Female|
|     Steve|   Rogers|102|     M| 9600.0|       Male|
|   Natasha|  Romanov|103|     F|22800.0|     Female|
|    Tyrion|Lannister|104|     M|13200.0|       Male|
+----------+---------+---+------+-------+-----------+



withColumnRenamed()

In [77]:
df = df.withColumnRenamed("id", "id_number")

In [78]:
df.show()

+----------+-----------+---------+---------+------+-------+-----------+
|first_name|middle_name|last_name|id_number|gender| salary|full_gender|
+----------+-----------+---------+---------+------+-------+-----------+
|  Jonathon|        Don|     Snow|      100|     M| 6000.0|       Male|
|      Arya|          K|    Stark|      101|     F|12000.0|     Female|
|     Steve|          S|   Rogers|      102|     M| 9600.0|       Male|
|   Natasha|          J|  Romanov|      103|     F|22800.0|     Female|
|    Tyrion|          K|Lannister|      104|     M|13200.0|       Male|
+----------+-----------+---------+---------+------+-------+-----------+



# Filter

In [79]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

In [80]:
data = [
        (("Jonathan","Don","Snow"),["Java","Scala","C++"],"OH","M"),
        (("Daniel","C","Targ"),["Java","Go","C++"],"NY","F"),
        (("Tyrion","K","Lannister"),["Python","Scala","C"],"NC","M"),
        (("Arya","F","Stark"),["JavaScript","HTML","CSS"],"WA","F"),
        (("Cersei","","Lannister"),["Python","Scala","Java"],"CA","M"),
        ]

In [81]:
schema = StructType([
                    StructField("name",StructType([
                                                    StructField("first_name",StringType(),True),
                                                    StructField("middle_name",StringType(),True),
                                                    StructField("last_name",StringType(),True)
                                                ]),True),
                    StructField("languages", ArrayType(StringType()),True),
                    StructField("state", StringType(),True),
                    StructField("gender", StringType(),True),
                    ])

In [82]:
df = spark.createDataFrame(data=data,schema=schema)

In [83]:
df.show(truncate=False)

+----------------------+-----------------------+-----+------+
|name                  |languages              |state|gender|
+----------------------+-----------------------+-----+------+
|{Jonathan, Don, Snow} |[Java, Scala, C++]     |OH   |M     |
|{Daniel, C, Targ}     |[Java, Go, C++]        |NY   |F     |
|{Tyrion, K, Lannister}|[Python, Scala, C]     |NC   |M     |
|{Arya, F, Stark}      |[JavaScript, HTML, CSS]|WA   |F     |
|{Cersei, , Lannister} |[Python, Scala, Java]  |CA   |M     |
+----------------------+-----------------------+-----+------+



In [84]:
df.printSchema()

root
 |-- name: struct (nullable = true)
 |    |-- first_name: string (nullable = true)
 |    |-- middle_name: string (nullable = true)
 |    |-- last_name: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)



In [85]:
df.filter(df.state == 'NC').show(truncate=False)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{Tyrion, K, Lannister}|[Python, Scala, C]|NC   |M     |
+----------------------+------------------+-----+------+



In [86]:
df.filter(df.state != 'NC').show(truncate=False)

+---------------------+-----------------------+-----+------+
|name                 |languages              |state|gender|
+---------------------+-----------------------+-----+------+
|{Jonathan, Don, Snow}|[Java, Scala, C++]     |OH   |M     |
|{Daniel, C, Targ}    |[Java, Go, C++]        |NY   |F     |
|{Arya, F, Stark}     |[JavaScript, HTML, CSS]|WA   |F     |
|{Cersei, , Lannister}|[Python, Scala, Java]  |CA   |M     |
+---------------------+-----------------------+-----+------+



In [87]:
df.filter((df.state != 'NC')&(df.gender == 'F')).show(truncate=False)

+-----------------+-----------------------+-----+------+
|name             |languages              |state|gender|
+-----------------+-----------------------+-----+------+
|{Daniel, C, Targ}|[Java, Go, C++]        |NY   |F     |
|{Arya, F, Stark} |[JavaScript, HTML, CSS]|WA   |F     |
+-----------------+-----------------------+-----+------+



In [88]:
df.filter(df.state.isin(["OH"])).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|{Jonathan, Don, S...|[Java, Scala, C++]|   OH|     M|
+--------------------+------------------+-----+------+



In [89]:
df.filter(df.state.startswith("N")).show(truncate=False)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{Daniel, C, Targ}     |[Java, Go, C++]   |NY   |F     |
|{Tyrion, K, Lannister}|[Python, Scala, C]|NC   |M     |
+----------------------+------------------+-----+------+



In [90]:
df.filter(df.state.like("N%")).show(truncate=False)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{Daniel, C, Targ}     |[Java, Go, C++]   |NY   |F     |
|{Tyrion, K, Lannister}|[Python, Scala, C]|NC   |M     |
+----------------------+------------------+-----+------+



In [91]:
from pyspark.sql.functions import array_contains

In [92]:
df.filter(array_contains(df.languages,"Python")).show(truncate=False)

+----------------------+---------------------+-----+------+
|name                  |languages            |state|gender|
+----------------------+---------------------+-----+------+
|{Tyrion, K, Lannister}|[Python, Scala, C]   |NC   |M     |
|{Cersei, , Lannister} |[Python, Scala, Java]|CA   |M     |
+----------------------+---------------------+-----+------+



# Distinct and DropDuplicates

In [93]:
columns = ["name","depatment","salary"]
data = [
        ("Jonathan","Sales",6500),\
        ("Arya","Developer",10000),\
        ("Tony","Developer",10000),\
        ("Bruce","Manager",15000),\
        ("Tony","Developer",10000),\
        ("Arya","Developer",10000),\
        ("Sansa","Sales",8000),\
        ("Jamie","Sales",6000),\
        ("Tyrion","Director",20000),\
        ("Jonathan","Sales",5000),\
        ]

In [94]:
df = spark.createDataFrame(data,columns)

In [95]:
df.show()

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|Jonathan|    Sales|  6500|
|    Arya|Developer| 10000|
|    Tony|Developer| 10000|
|   Bruce|  Manager| 15000|
|    Tony|Developer| 10000|
|    Arya|Developer| 10000|
|   Sansa|    Sales|  8000|
|   Jamie|    Sales|  6000|
|  Tyrion| Director| 20000|
|Jonathan|    Sales|  5000|
+--------+---------+------+



In [96]:
df.distinct().show() 
#This runs a distinct on all row object. 
#For a row to be eliminated due to distinct, it has to contain all the fieelds redundant

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|Jonathan|    Sales|  6500|
|   Sansa|    Sales|  8000|
|   Bruce|  Manager| 15000|
|Jonathan|    Sales|  5000|
|    Tony|Developer| 10000|
|    Arya|Developer| 10000|
|  Tyrion| Director| 20000|
|   Jamie|    Sales|  6000|
+--------+---------+------+



In [97]:
df.dropDuplicates().show()
#Same as distinct

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|Jonathan|    Sales|  6500|
|   Sansa|    Sales|  8000|
|   Bruce|  Manager| 15000|
|Jonathan|    Sales|  5000|
|    Tony|Developer| 10000|
|    Arya|Developer| 10000|
|  Tyrion| Director| 20000|
|   Jamie|    Sales|  6000|
+--------+---------+------+



In [98]:
df.dropDuplicates(subset=["salary","depatment"]).orderBy("name").show()

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|    Arya|Developer| 10000|
|   Bruce|  Manager| 15000|
|   Jamie|    Sales|  6000|
|Jonathan|    Sales|  6500|
|Jonathan|    Sales|  5000|
|   Sansa|    Sales|  8000|
|  Tyrion| Director| 20000|
+--------+---------+------+



# sort and orderBy

In [99]:
columns = ["name","depatment","salary"]
data = [
        ("Jonathan","Sales",6500),\
        ("Arya","Developer",10000),\
        ("Tony","Developer",10000),\
        ("Bruce","Manager",15000),\
        ("Tony","Developer",10000),\
        ("Arya","Developer",10000),\
        ("Sansa","Sales",8000),\
        ("Jamie","Sales",6000),\
        ("Tyrion","Director",20000),\
        ("Jonathan","Sales",5000),\
        ]

In [100]:
df = spark.createDataFrame(data,columns)

In [101]:
df.show()

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|Jonathan|    Sales|  6500|
|    Arya|Developer| 10000|
|    Tony|Developer| 10000|
|   Bruce|  Manager| 15000|
|    Tony|Developer| 10000|
|    Arya|Developer| 10000|
|   Sansa|    Sales|  8000|
|   Jamie|    Sales|  6000|
|  Tyrion| Director| 20000|
|Jonathan|    Sales|  5000|
+--------+---------+------+



In [102]:
df.sort(df.salary.desc(),df.name).show()

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|  Tyrion| Director| 20000|
|   Bruce|  Manager| 15000|
|    Arya|Developer| 10000|
|    Arya|Developer| 10000|
|    Tony|Developer| 10000|
|    Tony|Developer| 10000|
|   Sansa|    Sales|  8000|
|Jonathan|    Sales|  6500|
|   Jamie|    Sales|  6000|
|Jonathan|    Sales|  5000|
+--------+---------+------+



In [103]:
df.orderBy(df.salary.desc(),df.name).show()

+--------+---------+------+
|    name|depatment|salary|
+--------+---------+------+
|  Tyrion| Director| 20000|
|   Bruce|  Manager| 15000|
|    Arya|Developer| 10000|
|    Arya|Developer| 10000|
|    Tony|Developer| 10000|
|    Tony|Developer| 10000|
|   Sansa|    Sales|  8000|
|Jonathan|    Sales|  6500|
|   Jamie|    Sales|  6000|
|Jonathan|    Sales|  5000|
+--------+---------+------+



# groupBy

groupBy() collects identical rows into groups and performs an aggregation on a given column per grouped data.

When we perform groupBy() on PySpark Dataframe, it returns GroupedData object which contains below aggregate functions.

count() - Returns the count of rows for each group.<br>
mean() - Returns the mean of values for each group.<br>
max() - Returns the maximum of values for each group.<br> 
min() - Returns the minimum of values for each group.<br>
sum() - Returns the total for values for each group.<br>
avg() - Returns the average for values for each group.<br> 
agg() - Using agg() function, we can calculate more than one aggregate at a time.<br>
pivot() - This function is used to Pivot the DataFrame.

In [104]:
columns = ["employee_name","department","state","salary","age","bonus"]
data = [
        ("Jonathan","Developer","NC",90000,35,2000),\
        ("Tony","Developer","NY",120000,45,3000),\
        ("Arya","Developer","NC",95000,18,4000),\
        ("Bruce","Sales","CA",100000,58,1000),\
        ("Natasha","Sales","CA",80000,40,5000),\
        ("Steve","Sales","WA",70000,28,8000),\
        ("Thanos","Finance","NC",60000,75,3000),\
        ("Sansa","Developer","WA",90000,23,9000),\
        ("Gandalf","Finance","NY",150000,80,2000),\
        ("Cercei","Finance","NC",85000,46,2500),\
        ("Thor","Sales","WA",75000,50,5000),\
        ("Messi","Developer","NY",130000,34,6000),\
        ]

In [105]:
df = spark.createDataFrame(data,columns)

In [106]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|     Jonathan| Developer|   NC| 90000| 35| 2000|
|         Tony| Developer|   NY|120000| 45| 3000|
|         Arya| Developer|   NC| 95000| 18| 4000|
|        Bruce|     Sales|   CA|100000| 58| 1000|
|      Natasha|     Sales|   CA| 80000| 40| 5000|
|        Steve|     Sales|   WA| 70000| 28| 8000|
|       Thanos|   Finance|   NC| 60000| 75| 3000|
|        Sansa| Developer|   WA| 90000| 23| 9000|
|      Gandalf|   Finance|   NY|150000| 80| 2000|
|       Cercei|   Finance|   NC| 85000| 46| 2500|
|         Thor|     Sales|   WA| 75000| 50| 5000|
|        Messi| Developer|   NY|130000| 34| 6000|
+-------------+----------+-----+------+---+-----+



In [107]:
df.groupBy(df.department)

In [108]:
df.groupBy(df.department).max("salary").show() #do not pur df.salary -> does not take a Row() object. Instead a string

+----------+-----------+
|department|max(salary)|
+----------+-----------+
|     Sales|     100000|
|   Finance|     150000|
| Developer|     130000|
+----------+-----------+



In [109]:
df.groupBy(df.department).max("salary","age").show()

+----------+-----------+--------+
|department|max(salary)|max(age)|
+----------+-----------+--------+
|     Sales|     100000|      58|
|   Finance|     150000|      80|
| Developer|     130000|      45|
+----------+-----------+--------+



In [110]:
df.groupBy(df.department,df.state).mean().show()

+----------+-----+-----------+--------+----------+
|department|state|avg(salary)|avg(age)|avg(bonus)|
+----------+-----+-----------+--------+----------+
|   Finance|   NY|   150000.0|    80.0|    2000.0|
|     Sales|   CA|    90000.0|    49.0|    3000.0|
|     Sales|   WA|    72500.0|    39.0|    6500.0|
| Developer|   WA|    90000.0|    23.0|    9000.0|
|   Finance|   NC|    72500.0|    60.5|    2750.0|
| Developer|   NY|   125000.0|    39.5|    4500.0|
| Developer|   NC|    92500.0|    26.5|    3000.0|
+----------+-----+-----------+--------+----------+



In [111]:
df.groupBy(df.department,df.state).sum("salary","bonus").show()

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|   Finance|   NY|     150000|      2000|
|     Sales|   CA|     180000|      6000|
|     Sales|   WA|     145000|     13000|
| Developer|   WA|      90000|      9000|
|   Finance|   NC|     145000|      5500|
| Developer|   NY|     250000|      9000|
| Developer|   NC|     185000|      6000|
+----------+-----+-----------+----------+



In [112]:
from pyspark.sql.functions import sum,avg,max,mean,count

In [113]:
df.groupBy(df.department).agg(
                                sum("salary").alias("salary_sum"),\
                                max("salary").alias("salary_max"),\
                                sum("bonus").alias("bonus_sum"),\
                                max("bonus").alias("bonus_max"),\

                            ).show()

+----------+----------+----------+---------+---------+
|department|salary_sum|salary_max|bonus_sum|bonus_max|
+----------+----------+----------+---------+---------+
|     Sales|    325000|    100000|    19000|     8000|
|   Finance|    295000|    150000|     7500|     3000|
| Developer|    525000|    130000|    24000|     9000|
+----------+----------+----------+---------+---------+



In [114]:
df.groupBy(df.department).agg(
                                sum("salary").alias("salary_sum"),\
                                max("salary").alias("salary_max"),\
                                sum("bonus").alias("bonus_sum"),\
                                max("bonus").alias("bonus_max"),\

                            ).filter("bonus_max > 3000").show()

+----------+----------+----------+---------+---------+
|department|salary_sum|salary_max|bonus_sum|bonus_max|
+----------+----------+----------+---------+---------+
|     Sales|    325000|    100000|    19000|     8000|
| Developer|    525000|    130000|    24000|     9000|
+----------+----------+----------+---------+---------+



# Join

join(self,other,on=None,how=None) <br>
df1.join(df2.on="column_name",how="inner")

In [126]:
department_schema = ["department_id","department_name"]
department = [
            (10,"finance")
            ,(20,"sales")
            ,(30,"developer")
            ,(40,"hr")
            ,(50,"Admin")
            ]

In [144]:
employee_schema = ["emp_id","emp_name","emp_manager_id","year_of_joining", "emp_dept_id","gender","salary"]
employee = [
            (1,"Jonathan",15,"2019",30,"M",90000),
            (2,"Tyrion",5,"2015",40,"M",80000),
            (3,"Arya",10,"2020",20,"F",80000),
            (4,"Thanos",19,"2010",10,"M",110000),
            (5,"Tony",19,"2015",40,"M",150000),
            (6,"Danny",15,"2018",30,"F",70000),
            (7,"Natasha",4,"2019",10,"F",95000),
            (8,"Bruce",10,"2021",20,"M",60000),
            (9,"Sam",15,"2014",30,"M",80000),
            (10,"Cercei",19,"2012",20,"F",120000),
            (11,"Varis",5,"2016",40,"M",65000),
            (12,"Jamie",4,"2013",10,"M",95000),
            (13,"Wanda",10,"2017",20,"F",75000),
            (14,"Sansa",4,"2018",10,"F",100000),
            (15,"Loki",5,"2020",40,"M",70000),
            (16,"Ned",19,"2011",30,"M",160000),
            (17,"Cap_Marvel",19,"2021",60,"F",50000),
            (18,"Hound",17,"2021",60,"M",450000),
            (19,"Fury",0,"2010",0,"M",200000),
           ]

In [145]:
deptDF = spark.createDataFrame(data=department,schema=department_schema)

In [146]:
deptDF.show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|           10|        finance|
|           20|          sales|
|           30|      developer|
|           40|             hr|
|           50|          Admin|
+-------------+---------------+



In [147]:
empDF = spark.createDataFrame(data=employee,schema=employee_schema)

In [148]:
empDF.show()

+------+----------+--------------+---------------+-----------+------+------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|
+------+----------+--------------+---------------+-----------+------+------+
|     1|  Jonathan|            15|           2019|         30|     M| 90000|
|     2|    Tyrion|             5|           2015|         40|     M| 80000|
|     3|      Arya|            10|           2020|         20|     F| 80000|
|     4|    Thanos|            19|           2010|         10|     M|110000|
|     5|      Tony|            19|           2015|         40|     M|150000|
|     6|     Danny|            15|           2018|         30|     F| 70000|
|     7|   Natasha|             4|           2019|         10|     F| 95000|
|     8|     Bruce|            10|           2021|         20|     M| 60000|
|     9|       Sam|            15|           2014|         30|     M| 80000|
|    10|    Cercei|            19|           2012|         20|     F|120000|

Inner Join

In [152]:
empDF.join(deptDF,empDF.emp_dept_id==deptDF.department_id,how="inner").orderBy(empDF.emp_id).show()

+------+--------+--------------+---------------+-----------+------+------+-------------+---------------+
|emp_id|emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|department_id|department_name|
+------+--------+--------------+---------------+-----------+------+------+-------------+---------------+
|     1|Jonathan|            15|           2019|         30|     M| 90000|           30|      developer|
|     2|  Tyrion|             5|           2015|         40|     M| 80000|           40|             hr|
|     3|    Arya|            10|           2020|         20|     F| 80000|           20|          sales|
|     4|  Thanos|            19|           2010|         10|     M|110000|           10|        finance|
|     5|    Tony|            19|           2015|         40|     M|150000|           40|             hr|
|     6|   Danny|            15|           2018|         30|     F| 70000|           30|      developer|
|     7| Natasha|             4|           2019|       

Left Join

In [153]:
empDF.join(deptDF,empDF.emp_dept_id==deptDF.department_id,how="left").orderBy(empDF.emp_id).show()

+------+----------+--------------+---------------+-----------+------+------+-------------+---------------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|department_id|department_name|
+------+----------+--------------+---------------+-----------+------+------+-------------+---------------+
|     1|  Jonathan|            15|           2019|         30|     M| 90000|           30|      developer|
|     2|    Tyrion|             5|           2015|         40|     M| 80000|           40|             hr|
|     3|      Arya|            10|           2020|         20|     F| 80000|           20|          sales|
|     4|    Thanos|            19|           2010|         10|     M|110000|           10|        finance|
|     5|      Tony|            19|           2015|         40|     M|150000|           40|             hr|
|     6|     Danny|            15|           2018|         30|     F| 70000|           30|      developer|
|     7|   Natasha|             4|   

In [154]:
empDF.join(deptDF,empDF.emp_dept_id==deptDF.department_id,how="right").orderBy(empDF.emp_id).show()

+------+--------+--------------+---------------+-----------+------+------+-------------+---------------+
|emp_id|emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|department_id|department_name|
+------+--------+--------------+---------------+-----------+------+------+-------------+---------------+
|  null|    null|          null|           null|       null|  null|  null|           50|          Admin|
|     1|Jonathan|            15|           2019|         30|     M| 90000|           30|      developer|
|     2|  Tyrion|             5|           2015|         40|     M| 80000|           40|             hr|
|     3|    Arya|            10|           2020|         20|     F| 80000|           20|          sales|
|     4|  Thanos|            19|           2010|         10|     M|110000|           10|        finance|
|     5|    Tony|            19|           2015|         40|     M|150000|           40|             hr|
|     6|   Danny|            15|           2018|       

In [155]:
empDF.join(deptDF,empDF.emp_dept_id==deptDF.department_id,how="fullouter").orderBy(empDF.emp_id).show()

+------+----------+--------------+---------------+-----------+------+------+-------------+---------------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|department_id|department_name|
+------+----------+--------------+---------------+-----------+------+------+-------------+---------------+
|  null|      null|          null|           null|       null|  null|  null|           50|          Admin|
|     1|  Jonathan|            15|           2019|         30|     M| 90000|           30|      developer|
|     2|    Tyrion|             5|           2015|         40|     M| 80000|           40|             hr|
|     3|      Arya|            10|           2020|         20|     F| 80000|           20|          sales|
|     4|    Thanos|            19|           2010|         10|     M|110000|           10|        finance|
|     5|      Tony|            19|           2015|         40|     M|150000|           40|             hr|
|     6|     Danny|            15|   

In [156]:
empDF.join(deptDF,empDF.emp_dept_id==deptDF.department_id,how="leftanti").orderBy(empDF.emp_id).show()

+------+----------+--------------+---------------+-----------+------+------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|
+------+----------+--------------+---------------+-----------+------+------+
|    17|Cap_Marvel|            19|           2021|         60|     F| 50000|
|    18|     Hound|            17|           2021|         60|     M|450000|
|    19|      Fury|             0|           2010|          0|     M|200000|
+------+----------+--------------+---------------+-----------+------+------+



# Union

Merging 2 DataFrames with same schema

In [157]:
emp1_schema = ["emp_id","emp_name","emp_manager_id","year_of_joining", "emp_dept_id","gender","salary"]
emp1 = [
            (1,"Jonathan",15,"2019",30,"M",90000),
            (2,"Tyrion",5,"2015",40,"M",80000),
            (3,"Arya",10,"2020",20,"F",80000),
            (4,"Thanos",19,"2010",10,"M",110000),
            (5,"Tony",19,"2015",40,"M",150000),
            (6,"Danny",15,"2018",30,"F",70000),
            (7,"Natasha",4,"2019",10,"F",95000),
            (8,"Bruce",10,"2021",20,"M",60000),
            (9,"Sam",15,"2014",30,"M",80000),
        ]

emp2_schema = ["emp_id","emp_name","emp_manager_id","year_of_joining", "emp_dept_id","gender","salary"]
emp2= [
            (10,"Cercei",19,"2012",20,"F",120000),
            (11,"Varis",5,"2016",40,"M",65000),
            (12,"Jamie",4,"2013",10,"M",95000),
            (13,"Wanda",10,"2017",20,"F",75000),
            (14,"Sansa",4,"2018",10,"F",100000),
            (15,"Loki",5,"2020",40,"M",70000),
            (16,"Ned",19,"2011",30,"M",160000),
            (17,"Cap_Marvel",19,"2021",60,"F",50000),
            (18,"Hound",17,"2021",60,"M",450000),
            (19,"Fury",0,"2010",0,"M",200000),
        ]

In [159]:
emp1DF = spark.createDataFrame(emp1,emp1_schema)
emp2DF = spark.createDataFrame(emp2,emp2_schema)

In [160]:
emp1DF.show()

+------+--------+--------------+---------------+-----------+------+------+
|emp_id|emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|
+------+--------+--------------+---------------+-----------+------+------+
|     1|Jonathan|            15|           2019|         30|     M| 90000|
|     2|  Tyrion|             5|           2015|         40|     M| 80000|
|     3|    Arya|            10|           2020|         20|     F| 80000|
|     4|  Thanos|            19|           2010|         10|     M|110000|
|     5|    Tony|            19|           2015|         40|     M|150000|
|     6|   Danny|            15|           2018|         30|     F| 70000|
|     7| Natasha|             4|           2019|         10|     F| 95000|
|     8|   Bruce|            10|           2021|         20|     M| 60000|
|     9|     Sam|            15|           2014|         30|     M| 80000|
+------+--------+--------------+---------------+-----------+------+------+



In [161]:
emp2DF.show()

+------+----------+--------------+---------------+-----------+------+------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|
+------+----------+--------------+---------------+-----------+------+------+
|    10|    Cercei|            19|           2012|         20|     F|120000|
|    11|     Varis|             5|           2016|         40|     M| 65000|
|    12|     Jamie|             4|           2013|         10|     M| 95000|
|    13|     Wanda|            10|           2017|         20|     F| 75000|
|    14|     Sansa|             4|           2018|         10|     F|100000|
|    15|      Loki|             5|           2020|         40|     M| 70000|
|    16|       Ned|            19|           2011|         30|     M|160000|
|    17|Cap_Marvel|            19|           2021|         60|     F| 50000|
|    18|     Hound|            17|           2021|         60|     M|450000|
|    19|      Fury|             0|           2010|          0|     M|200000|

In [163]:
emp1DF.union(emp2DF).show()

+------+----------+--------------+---------------+-----------+------+------+
|emp_id|  emp_name|emp_manager_id|year_of_joining|emp_dept_id|gender|salary|
+------+----------+--------------+---------------+-----------+------+------+
|     1|  Jonathan|            15|           2019|         30|     M| 90000|
|     2|    Tyrion|             5|           2015|         40|     M| 80000|
|     3|      Arya|            10|           2020|         20|     F| 80000|
|     4|    Thanos|            19|           2010|         10|     M|110000|
|     5|      Tony|            19|           2015|         40|     M|150000|
|     6|     Danny|            15|           2018|         30|     F| 70000|
|     7|   Natasha|             4|           2019|         10|     F| 95000|
|     8|     Bruce|            10|           2021|         20|     M| 60000|
|     9|       Sam|            15|           2014|         30|     M| 80000|
|    10|    Cercei|            19|           2012|         20|     F|120000|

# unionByName

In [164]:
data1_schema = ["id","name","dept","salary"]
data1 = [
    (1,"Jon","sales",5000),
    (2,"Danny","finance",6000),
    (3,"Arya","admin",4000),
    (4,"Ned","sales",7000)
]

In [165]:
data2_schema = ["id","name","dept","join_year"]
data2 = [
    (1,"Jon","sales","2019"),
    (2,"Danny","finance","2018"),
    (3,"Arya","admin","2020"),
    (4,"Ned","sales","2010")
]

In [166]:
df1 = spark.createDataFrame(data1,data1_schema)
df2 = spark.createDataFrame(data2,data2_schema)

In [169]:
print(df1.show())
print(df2.show())

+---+-----+-------+------+
| id| name|   dept|salary|
+---+-----+-------+------+
|  1|  Jon|  sales|  5000|
|  2|Danny|finance|  6000|
|  3| Arya|  admin|  4000|
|  4|  Ned|  sales|  7000|
+---+-----+-------+------+

None
+---+-----+-------+---------+
| id| name|   dept|join_year|
+---+-----+-------+---------+
|  1|  Jon|  sales|     2019|
|  2|Danny|finance|     2018|
|  3| Arya|  admin|     2020|
|  4|  Ned|  sales|     2010|
+---+-----+-------+---------+

None


In [170]:
df1.union(df2).show()

+---+-----+-------+------+
| id| name|   dept|salary|
+---+-----+-------+------+
|  1|  Jon|  sales|  5000|
|  2|Danny|finance|  6000|
|  3| Arya|  admin|  4000|
|  4|  Ned|  sales|  7000|
|  1|  Jon|  sales|  2019|
|  2|Danny|finance|  2018|
|  3| Arya|  admin|  2020|
|  4|  Ned|  sales|  2010|
+---+-----+-------+------+



In [172]:
df2.union(df1).show()

+---+-----+-------+---------+
| id| name|   dept|join_year|
+---+-----+-------+---------+
|  1|  Jon|  sales|     2019|
|  2|Danny|finance|     2018|
|  3| Arya|  admin|     2020|
|  4|  Ned|  sales|     2010|
|  1|  Jon|  sales|     5000|
|  2|Danny|finance|     6000|
|  3| Arya|  admin|     4000|
|  4|  Ned|  sales|     7000|
+---+-----+-------+---------+



In [175]:
df1.unionByName(df2,allowMissingColumns=True).show()

+---+-----+-------+------+---------+
| id| name|   dept|salary|join_year|
+---+-----+-------+------+---------+
|  1|  Jon|  sales|  5000|     null|
|  2|Danny|finance|  6000|     null|
|  3| Arya|  admin|  4000|     null|
|  4|  Ned|  sales|  7000|     null|
|  1|  Jon|  sales|  null|     2019|
|  2|Danny|finance|  null|     2018|
|  3| Arya|  admin|  null|     2020|
|  4|  Ned|  sales|  null|     2010|
+---+-----+-------+------+---------+



# UDF

UDFs - User Defined Functions Python functions that are not covered by existing Pyspark functions.<br>
<br>
UDFs need to be created using udf() function in order to use as a Pyspark function.<br>
UDFs must be registered using udf.register() function in order to be used in Pyspark SQL.<br>
<br>
Performance of UDFs are not good as for Pyspark, UDFs are a black box and Pyspark Optimization techniques cannot be applied on UDFs.

In [221]:
schema = ["id","first_name","last_name","dept","salary"]
data = [
    (1,"jon","Snow","sales",5000),
    (2,"danny","Targy","finance",6000),
    (3,"arya","Stark","admin",4000),
    (4,"ned","Stark","sales",7000)
]

In [236]:
df = spark.createDataFrame(data,schema)

In [237]:
df.show()

+---+----------+---------+-------+------+
| id|first_name|last_name|   dept|salary|
+---+----------+---------+-------+------+
|  1|       jon|     Snow|  sales|  5000|
|  2|     danny|    Targy|finance|  6000|
|  3|      arya|    Stark|  admin|  4000|
|  4|       ned|    Stark|  sales|  7000|
+---+----------+---------+-------+------+



In [224]:
#Requirement - 
# 1. change first letter of first_name into uppercase

In [238]:
def convert_case(word):
    '''Takes a word as input and returns the word with first letter in upper case
    Example: convert_case("jon") returns Jon'''
    return word[0].upper() + word[1:]

In [239]:
from pyspark.sql.functions import udf, StringType

In [227]:
convert_case_udf_for_Pyspark = udf(lambda word: convert_case(word), StringType())

In [228]:
convert_case_udf()

Column<'<lambda>()'>

In [229]:
# Using UDF in Pyspark

In [230]:
df = df.select(
        df.id,
        convert_case_udf_for_Pyspark(df.first_name).alias("fname_caps"),
        df.last_name,
        df.dept,
        df.salary
        )

In [231]:
df.show()

+---+----------+---------+-------+------+
| id|fname_caps|last_name|   dept|salary|
+---+----------+---------+-------+------+
|  1|       Jon|     Snow|  sales|  5000|
|  2|     Danny|    Targy|finance|  6000|
|  3|      Arya|    Stark|  admin|  4000|
|  4|       Ned|    Stark|  sales|  7000|
+---+----------+---------+-------+------+



Registering UDF: IN order to use Python functions in Pyspark SQL, we need to register UDF.<br>
UDF Registration:<br>
spark.udf.register(UDF_name,Python_function,returnType=None) -> Register a Python function (including lambda function) or a user-defined function
as a SQL function.

In [240]:
spark.udf.register("convert_case_udf_for_sql",convert_case,returnType=StringType())

2021-09-21 13:21:12,596 WARN analysis.SimpleFunctionRegistry: The function convert_case_udf_for_sql replaced a previously registered function.


<function __main__.convert_case(word)>

In [241]:
df.show()

+---+----------+---------+-------+------+
| id|first_name|last_name|   dept|salary|
+---+----------+---------+-------+------+
|  1|       jon|     Snow|  sales|  5000|
|  2|     danny|    Targy|finance|  6000|
|  3|      arya|    Stark|  admin|  4000|
|  4|       ned|    Stark|  sales|  7000|
+---+----------+---------+-------+------+



In [242]:
df.createOrReplaceTempView("temp_table")

In [252]:
spark.sql("select * from temp_table").show()

+---+----------+---------+-------+------+
| id|first_name|last_name|   dept|salary|
+---+----------+---------+-------+------+
|  1|       jon|     Snow|  sales|  5000|
|  2|     danny|    Targy|finance|  6000|
|  3|      arya|    Stark|  admin|  4000|
|  4|       ned|    Stark|  sales|  7000|
+---+----------+---------+-------+------+



In [255]:
spark.sql("select id,convert_case_udf_for_sql(first_name) as first_name_conv,last_name,dept,salary from temp_table").show()

+---+---------------+---------+-------+------+
| id|first_name_conv|last_name|   dept|salary|
+---+---------------+---------+-------+------+
|  1|            Jon|     Snow|  sales|  5000|
|  2|          Danny|    Targy|finance|  6000|
|  3|           Arya|    Stark|  admin|  4000|
|  4|            Ned|    Stark|  sales|  7000|
+---+---------------+---------+-------+------+



In [256]:
df.show()

+---+----------+---------+-------+------+
| id|first_name|last_name|   dept|salary|
+---+----------+---------+-------+------+
|  1|       jon|     Snow|  sales|  5000|
|  2|     danny|    Targy|finance|  6000|
|  3|      arya|    Stark|  admin|  4000|
|  4|       ned|    Stark|  sales|  7000|
+---+----------+---------+-------+------+



# partitionBy()

In [262]:
from pyspark.sql import DataFrameWriter

In [259]:
columns = ["employee_name","department","state","salary","age","bonus"]
data = [
        ("Jonathan","Developer","NC",90000,35,2000),\
        ("Tony","Developer","NY",120000,45,3000),\
        ("Arya","Developer","NC",95000,18,4000),\
        ("Bruce","Sales","CA",100000,58,1000),\
        ("Natasha","Sales","CA",80000,40,5000),\
        ("Steve","Sales","WA",70000,28,8000),\
        ("Thanos","Finance","NC",60000,75,3000),\
        ("Sansa","Developer","WA",90000,23,9000),\
        ("Gandalf","Finance","NY",150000,80,2000),\
        ("Cercei","Finance","NC",85000,46,2500),\
        ("Thor","Sales","WA",75000,50,5000),\
        ("Messi","Developer","NY",130000,34,6000),\
        ]

In [260]:
df = spark.createDataFrame(data,columns)

In [261]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|     Jonathan| Developer|   NC| 90000| 35| 2000|
|         Tony| Developer|   NY|120000| 45| 3000|
|         Arya| Developer|   NC| 95000| 18| 4000|
|        Bruce|     Sales|   CA|100000| 58| 1000|
|      Natasha|     Sales|   CA| 80000| 40| 5000|
|        Steve|     Sales|   WA| 70000| 28| 8000|
|       Thanos|   Finance|   NC| 60000| 75| 3000|
|        Sansa| Developer|   WA| 90000| 23| 9000|
|      Gandalf|   Finance|   NY|150000| 80| 2000|
|       Cercei|   Finance|   NC| 85000| 46| 2500|
|         Thor|     Sales|   WA| 75000| 50| 5000|
|        Messi| Developer|   NY|130000| 34| 6000|
+-------------+----------+-----+------+---+-----+



In [264]:
df.write.option("header",True).partitionBy("state").csv("/sparkdata/partitionby")

In [ ]:
#4 directories will form in the mentioned file location